Large Scale Data Analysis - on movie reviews.   Siddhesh Chavan

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

In [ ]:
print("JAVA_HOME:", os.environ.get("JAVA_HOME"))
print("SPARK_HOME:", os.environ.get("SPARK_HOME"))


JAVA_HOME: /usr/lib/jvm/java-8-openjdk-amd64
SPARK_HOME: spark-3.2.0-bin-hadoop3.2


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
positive_words_RDD = sc.textFile('/content/gdrive/MyDrive/cloud/pos.txt', use_unicode=True)

In [ ]:
Map_Pos_Words = positive_words_RDD.map(lambda x: (x, 1)).collectAsMap()

In [ ]:
negative_words_RDD = sc.textFile('/content/gdrive/MyDrive/cloud/neg.txt', use_unicode=True)

In [ ]:
Map_Neg_Words = negative_words_RDD.map(lambda x: (x, 1)).collectAsMap()

In [ ]:
movie_reviews_RDD = sc.wholeTextFiles('/content/gdrive/MyDrive/cloud/moviereviews', use_unicode=True)

In [ ]:
pip install nltk


In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!pip install textblob
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob



In [ ]:
stop_words = set(stopwords.words('english'))
def preprocess(data):
    s = re.sub('\W+',' ', data.lower())
    s=TextBlob(s)
    s = s.tokenize()
    s = [w for w in s if not w in stop_words]
    s = [re.sub(r'[^aA-zZ]+', ' ', w) for w in s if len(w)>1]
    s = [re.sub(r'\W+', ' ', w) for w in s if len(w)>1]
    return (data,s)

In [ ]:
reviews_RDD = movie_reviews_RDD.map(lambda x: (x[0].split("/")[-1], preprocess(x[1])))

In [ ]:
def Senti_analysis(x, Map_Pos_Words, Map_Neg_Words):
  filename = x[0]
  original = x[1][0]
  words = x[1][1]
  positive_sc = 0
  negative_sc = 0

  for word in words:
    # Calculating +ve score. . . . . . .
    if word in Map_Pos_Words:
      positive_sc = positive_sc + 1
    #Calculating -ve score. . . . . . .
    if word in Map_Neg_Words:
      negative_sc = negative_sc + 1

  sentiment = "positive" if positive_sc > negative_sc else "negative"

  return (filename, original, words, positive_sc, negative_sc, sentiment)

In [ ]:
reviews_RDD_with_score = reviews_RDD.map(lambda x: Senti_analysis(x, Map_Pos_Words, Map_Neg_Words))

In [ ]:
reviews_sentiment_df = reviews_RDD_with_score.toDF(["filename", "review", "cleaned review", "positive_score", "negative_score", "sentiment"])

In [ ]:
reviews_sentiment_df.show()

+---------------+--------------------+--------------------+--------------+--------------+---------+
|       filename|              review|      cleaned review|positive_score|negative_score|sentiment|
+---------------+--------------------+--------------------+--------------+--------------+---------+
|cv744_10091.txt|talk about beatin...|[talk, beating, d...|            19|            37| negative|
| cv754_7709.txt|capsule : godawfu...|[capsule, godawfu...|            16|            36| negative|
|cv838_25886.txt|in our time . \ni...|[time, modern, wo...|            43|            33| positive|
|cv523_18285.txt|1989's " major le...|[major, league, d...|             6|            10| negative|
|cv401_13758.txt|i'm not sure who ...|[sure, genius, ca...|            49|            52| negative|
|cv131_11568.txt|about an hour or ...|[hour, jackal, ch...|            50|            46| positive|
|cv948_25870.txt|and just when you...|[thought, joblo, ...|            17|            23| negative|
